# Data Exploration: Southwest Airlines OTP and Load Factor Analysis

This notebook explores raw data from the **Bureau of Transportation Statistics (BTS)**, specifically:

- **Load Factor data** (2020–2025, annual files)  
- **On-Time Performance (OTP) data** (2020–2025, monthly files)

**Goal:**  
Understand the **structure**, **quality**, and **characteristics** of the data to prepare for further analysis.

---

## Import Libraries and Setup


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('..')

from src import DataLoader, DataCleaner

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# %matplotlib inline

---


### Cell 1. Load Raw Data

In [ ]:
# Initialize loader
loader = DataLoader(base_path='../data/raw')

# Load both datasets
lf_raw, otp_raw = loader.load_all_data()

---


### Cell 2. Explore Load Factor Data

In [ ]:
print("Load Factor Data Shape:", lf_raw.shape)
print("\nColumn Names and Types:")
print(lf_raw.dtypes)
print("\nFirst few rows:")
lf_raw.head()


# Check for missing values
print("Missing Values:")
print(lf_raw.isnull().sum())
print(f"\nPercentage missing:")
print((lf_raw.isnull().sum() / len(lf_raw) * 100).round(2))


# Summary statistics
lf_raw.describe()


# Check unique carriers
print(f"Number of unique carriers: {lf_raw['CARRIER'].nunique()}")
print("\nTop 10 carriers by record count:")
print(lf_raw['CARRIER'].value_counts().head(10))
print(f"\nSouthwest (WN) records: {len(lf_raw[lf_raw['CARRIER'] == 'WN']):,}")


# Date range in data
print("Year range:", lf_raw['YEAR'].min(), "-", lf_raw['YEAR'].max())
print("Month range:", lf_raw['MONTH'].min(), "-", lf_raw['MONTH'].max())



---


### Cell 3. Explore OTP Data

In [ ]:
print("OTP Data Shape:", otp_raw.shape)
print("\nColumn Names and Types:")
print(otp_raw.dtypes)
print("\nFirst few rows:")
otp_raw.head()


# Check for missing values
print("Missing Values:")
print(otp_raw.isnull().sum())
print(f"\nPercentage missing:")
print((otp_raw.isnull().sum() / len(otp_raw) * 100).round(2))


# Summary statistics
otp_raw.describe()


# Check unique carriers
print(f"Number of unique carriers: {otp_raw['OP_UNIQUE_CARRIER'].nunique()}")
print("\nTop 10 carriers by record count:")
print(otp_raw['OP_UNIQUE_CARRIER'].value_counts().head(10))
print(f"\nSouthwest (WN) records: {len(otp_raw[otp_raw['OP_UNIQUE_CARRIER'] == 'WN']):,}")


# Date range in data
print("Year range:", otp_raw['YEAR'].min(), "-", otp_raw['YEAR'].max())
print("Month range:", otp_raw['MONTH'].min(), "-", otp_raw['MONTH'].max())


---


### Cell 4. Clean Data and Filter for Southwest

In [ ]:
# Initialize cleaner for Southwest (WN)
cleaner = DataCleaner(carrier_code='WN')

# Clean load factor data
lf_clean = cleaner.clean_load_factor_data(lf_raw)
print("\n" + "="*60)

# Clean OTP data
otp_clean = cleaner.clean_otp_data(otp_raw)

---


### Cell 5. Explore Southwest-Specific Data

In [ ]:
# Check date coverage for Southwest
print("Southwest Load Factor Data:")
print(f"Date range: {lf_clean['DATE'].min()} to {lf_clean['DATE'].max()}")
print(f"Number of unique routes: {lf_clean['ROUTE'].nunique()}")
print(f"Number of unique origins: {lf_clean['ORIGIN'].nunique()}")
print(f"Number of unique destinations: {lf_clean['DEST'].nunique()}")

print("\nSouthwest OTP Data:")
print(f"Date range: {otp_clean['DATE'].min()} to {otp_clean['DATE'].max()}")
print(f"Number of unique routes: {otp_clean['ROUTE'].nunique()}")
print(f"Number of unique origins: {otp_clean['ORIGIN'].nunique()}")
print(f"Number of unique destinations: {otp_clean['DEST'].nunique()}")


# Calculate basic load factor for Southwest
overall_lf = (lf_clean['PASSENGERS'].sum() / lf_clean['SEATS'].sum() * 100)
print(f"Southwest Overall Load Factor (2020-2025): {overall_lf:.2f}%")


# Load factor by year
lf_by_year = lf_clean.groupby('YEAR').apply(
    lambda x: (x['PASSENGERS'].sum() / x['SEATS'].sum() * 100)
)

print("Load Factor by Year:")
print(lf_by_year)


# Visualize load factor trend
monthly_lf = lf_clean.groupby('DATE').apply(
    lambda x: (x['PASSENGERS'].sum() / x['SEATS'].sum() * 100)
).reset_index()
monthly_lf.columns = ['DATE', 'LOAD_FACTOR']

plt.figure(figsize=(14, 6))
plt.plot(monthly_lf['DATE'], monthly_lf['LOAD_FACTOR'], marker='o', linewidth=2)
plt.title('Southwest Airlines Load Factor Over Time (2020-2025)', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Load Factor (%)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


# Calculate basic OTP metrics
total_flights = len(otp_clean)
dep_ontime_pct = (1 - otp_clean['DEP_DEL15'].sum() / total_flights) * 100
arr_ontime_pct = (1 - otp_clean['ARR_DEL15'].sum() / total_flights) * 100
cancel_pct = (otp_clean['CANCELLED'].sum() / total_flights) * 100

print(f"Southwest Overall OTP Metrics (2020-2025):")
print(f"Total flights: {total_flights:,}")
print(f"Departure On-Time %: {dep_ontime_pct:.2f}%")
print(f"Arrival On-Time %: {arr_ontime_pct:.2f}%")
print(f"Cancellation %: {cancel_pct:.2f}%")


# OTP by year
otp_by_year = otp_clean.groupby('YEAR').apply(
    lambda x: pd.Series({
        'flights': len(x),
        'dep_ontime_pct': (1 - x['DEP_DEL15'].sum() / len(x)) * 100,
        'arr_ontime_pct': (1 - x['ARR_DEL15'].sum() / len(x)) * 100,
        'cancel_pct': (x['CANCELLED'].sum() / len(x)) * 100
    })
)

print("\nOTP Metrics by Year:")
print(otp_by_year)


# Visualize OTP trend
monthly_otp = otp_clean.groupby('DATE').apply(
    lambda x: pd.Series({
        'dep_ontime_pct': (1 - x['DEP_DEL15'].sum() / len(x)) * 100,
        'arr_ontime_pct': (1 - x['ARR_DEL15'].sum() / len(x)) * 100
    })
).reset_index()

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(monthly_otp['DATE'], monthly_otp['dep_ontime_pct'], 
        marker='o', label='Departure On-Time %', linewidth=2)
ax.plot(monthly_otp['DATE'], monthly_otp['arr_ontime_pct'], 
        marker='s', label='Arrival On-Time %', linewidth=2)
ax.set_title('Southwest Airlines On-Time Performance Over Time (2020-2025)', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('On-Time Percentage (%)', fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---


### Cell 6. Save Cleaned Data

In [ ]:
# Save cleaned data for use in other notebooks
import os
lf_path = '../data/processed/lf_clean_southwest.csv'
otp_path = '../data/processed/otp_clean_southwest.csv'

if os.path.exists(lf_path) and os.path.exists(otp_path):
    print("Cleaned data files already exist in data/processed/. Skipping save.")
else:
    lf_clean.to_csv(lf_path, index=False)
    otp_clean.to_csv(otp_path, index=False)
    print("Cleaned data saved to data/processed/")

---


## Key Findings from Exploration

1. **Data Quality**:  
   Both datasets provide robust coverage of Southwest Airlines from 2020 to 2025.

2. **Load Factor Trends**:  
   Observable patterns over time, likely reflecting the impact of COVID-19 and subsequent recovery.

3. **OTP Trends**:  
   On-time performance metrics display variations across the time period.

4. **Next Steps**:  
   - Identify the top 5 routes by passenger volume.
   - Analyze the relationship between load factor and OTP for these routes.


---


### Please move onto Notebook 2, ' `02_route_analysis.ipynb` '